In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import math
import runpy
import sys

In [2]:
if 'src' in sys.path:
    pass
else:
    sys.path.insert(0, 'src')

In [3]:
from infection import *
from classroom import *

In [8]:
# Infection Math 

# Close Range

# Shared Room

# Long Range
t_rate_dict = {
    'Simulation #': np.array([]),
    'Day #': np.array([]),
    'Step #': np.array([]),
    'Minute #': np.array([]),
    'Infected ID': np.array([]),
    'Other ID': np.array([]),
    'Close Range Transmission': np.array([]),
    'Shared Room Transmission': np.array([]),
    'Far Range Transmission': np.array([]),
    'Transmission by Minute': np.array([]),
    'Transmission by Step': np.array([]),
    'Transmission by Day': np.array([]),
    'Transmission by Sim': np.array([]),
}
df = pd.DataFrame.from_dict(t_rate_dict)
df

,Simulation #,Day #,Step #,Minute #,Infected ID,Other ID,Close Range Transmission,Shared Room Transmission,Far Range Transmission,Transmission by Minute,Transmission by Step,Transmission by Day,Transmission by Sim


In [5]:
# Air Flow Proxy

# conc_arr = [step][y][x][proportion]

In [6]:
%run -m av_run

KeyError: 'door_location'

c:\users\bailey\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: Unknown failure executing module: <av_run>
  warn('Unknown failure executing module: <%s>' % mod_name)
